**Library import**

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

**Obtain list with links of the sites to scrape**

In [2]:
# dataset location
male_path = r"C:\Users\BrechtDewilde\Documents\Github\Predicting-the-olympic-games\marathon\Data c&c\R\Data\marathonM.csv"
female_path = r"C:\Users\BrechtDewilde\Documents\Github\Predicting-the-olympic-games\marathon\Data c&c\R\Data\marathonF.csv"

In [6]:
# Male dataset creation
mlink = "https://www.worldathletics.org/athletes/search?query={}&countryCode={}&disciplineCode=&gender=male&environment="
males = pd.read_csv(male_path, index_col = 0)
males = males.drop("NOC", axis = 1)
males = males.drop_duplicates()

In [8]:
# Female link generation
flink = "https://www.worldathletics.org/athletes/search?query={}&countryCode={}&disciplineCode=&gender=female&environment="
females = pd.read_csv(female_path, index_col = 0)
females = females.drop_duplicates()
females = females.drop("NOC", axis = 1)

**Obtain sites country codes**

In [20]:
link = "https://www.worldathletics.org/athletes/search?query=Michael%2BShelley&countryCode=&disciplineCode=&gender=&environment="
site = requests.get(link).content
soup = BeautifulSoup(site, "html.parser")
countries = soup.find("select", {"name":"countryCode"}).find_all("option")

nationalities = []
labels = []
for label in countries:
    labels.append(label.attrs["value"])
    nationalities.append(label.text)

label_males = []
label_females = []
for i in males["Nationality"]:
    if i in nationalities:
        label_males.append(labels[nationalities.index(i)])
    else:
        label_males.append("")

for i in females["Nationality"]:
    if i in nationalities:
        label_females.append(labels[nationalities.index(i)])
    else:
        label_females.append("")      

**Add QueryName and QueryLabel to dataframes**

In [21]:
males["QueryName"] = ["+".join(i.split()) for i in males["Name"]]
males["QueryLabel"] = label_males
females["QueryName"] = ["+".join(i.split()) for i in females["Name"]]
females["QueryLabel"] = label_females

C:\Program Files (x86)\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Program Files (x86)\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


**Obtain search links**

In [23]:
mlinks = [mlink.format(list(males["QueryName"])[i], list(males["QueryLabel"])[i]) for i in range(males.shape[0])]
flinks = [flink.format(list(females["QueryName"])[i], list(females["QueryLabel"])[i]) for i in range(females.shape[0])]

**Obtain the link to each profile athletes page**

In [32]:
male_profiles = []
female_profiles = []
mnone_profiles = []
fnone_profiles = []
blink = "https://www.worldathletics.org/athletes/"
for link in mlinks:
    site = requests.get(link).content
    soup = BeautifulSoup(site, "html.parser")
    try:
        male_profiles.append(blink + soup.find("table", {"class": "records-table"}).find("a").attrs["href"])
    except:
        male_profiles.append("null")
        mnone_profiles.append(link)
    
for link in flinks:
    site = requests.get(link).content
    soup = BeautifulSoup(site, "html.parser")
    try:
        female_profiles.append(blink + soup.find("table", {"class": "records-table"}).find("a").attrs["href"])
    except:
        female_profiles.append("null")
        fnone_profiles.append(link)

female_profiles_backup = female_profiles
male_profiles_backup = male_profiles

Add manually the links of the non profile sites

In [40]:
missingM_links = ["https://www.worldathletics.org/athletes/azerbaijan/evans-kiplagat-250644", 
"https://www.worldathletics.org/athletes/burundi/tharcisse-gashaka-014177810",
"https://www.worldathletics.org/athletes/burundi/joachim-nshimirimana-180569",
"https://www.worldathletics.org/athletes/cambodia/kuniaki-takizaki-313331",
"https://www.worldathletics.org/athletes/central-african-republic/ferdinand-amadi-014432563",
"https://www.worldathletics.org/athletes/indonesia/ethel-hudson-11986",
"https://www.worldathletics.org/athletes/israel/zohar-zimro-258676",
"https://www.worldathletics.org/athletes/lesotho/gabashane-rakabaele-014354671",
"https://www.worldathletics.org/athletes/malawi/george-mambosasa-014430255",
"https://www.worldathletics.org/athletes/namibia/mynhardt-mbeumuna-kawanivi-280028",
"https://www.worldathletics.org/athletes/nepal/baikuntha-manandhar-6434",
"https://www.worldathletics.org/athletes/rwanda/telesphore-dusabe-014432534",
"https://www.worldathletics.org/athletes/rwanda/mathias-ntawulikura-2084",
"https://www.worldathletics.org/athletes/zambia/trev-haynes-014552623",
"https://www.worldathletics.org/athletes/zimbabwe/cutbert-nyasango-180114"]

m_indices = [i for i, x in enumerate(male_profiles) if x == "null"]
missingF_links = ["https://www.worldathletics.org/athletes/germany/luminita-zaituc-66785", "https://www.worldathletics.org/athletes/honduras/gina-coello-60331", "https://www.worldathletics.org/athletes/lesotho/mamorallo-tjoka-184788", "https://www.worldathletics.org/athletes/madagascar/clarisse-rasoarizay-76065", "https://www.worldathletics.org/athletes/malawi/tereza-master-200505", "https://www.worldathletics.org/athletes/namibia/beata-naigambo-138596", "https://www.worldathletics.org/athletes/rwanda/claudette-mukasakindi-233268"]
f_indices = [i for i, x in enumerate(female_profiles) if x == "null"]

In [41]:
for index, profile in zip(m_indices, missingM_links):
    male_profiles[index] = profile

for index, profile in zip(f_indices, missingF_links):
    female_profiles[index] = profile

**Scrape the progression page of each profile**

Males

In [147]:
athlete = []
year = []
performance = []
place = []
date = []
for name, link in zip(males["Name"], male_profiles):
    # does the link contain a //athletes remove this
    if "athletes//" in link:
        link = link.replace("athletes//", "")
        
    site = requests.get(link).content
    soup = BeautifulSoup(site, "html.parser")
    
    counter = 0
    for i in soup.find_all("div", {"class":"row table-header"}):
        if "Marathon" in i.text and "Half" not in i.text:
            table = i.parent.find("table", {"class":"records-table"})
            for y in table.find_all("td", {"data-th":" "}):
                year.append(y.text.strip())
                counter += 1
            for p in table.find_all("td", {"data-th":"Performance"}):
                performance.append(p.text.strip())
            for pl in table.find_all("td", {"data-th":"Place"}):
                place.append(pl.text.strip())
            for d in table.find_all("td", {"data-th":"Date"}):
                date.append(d.text.strip())
            athlete.extend([name for n in range(counter)])
            break

In [148]:
df = pd.DataFrame({"athlete": athlete, "year": year, "performance": performance, "place":place, "date":date})
df.to_csv(r"C:\Users\BrechtDewilde\Documents\Github\Predicting-the-olympic-games\marathon\Data c&c\R\Data\record history\male_records.csv", index = False)

Females

In [144]:
athlete = []
year = []
performance = []
place = []
date = []
for name, link in zip(females["Name"], female_profiles):
    # does the link contain a //athletes remove this
    if "athletes//" in link:
        link = link.replace("athletes//", "")
        
    site = requests.get(link).content
    soup = BeautifulSoup(site, "html.parser")
    
    counter = 0
    for i in soup.find_all("div", {"class":"row table-header"}):
        if "Marathon" in i.text and "Half" not in i.text:
            table = i.parent.find("table", {"class":"records-table"})
            for y in table.find_all("td", {"data-th":" "}):
                year.append(y.text.strip())
                counter += 1
            for p in table.find_all("td", {"data-th":"Performance"}):
                performance.append(p.text.strip())
            for pl in table.find_all("td", {"data-th":"Place"}):
                place.append(pl.text.strip())
            for d in table.find_all("td", {"data-th":"Date"}):
                date.append(d.text.strip())
            athlete.extend([name for n in range(counter)])
            break

athlete https://www.worldathletics.org/athletes/athlete=115604, year 8, perf 8, place 8, date 8
athlete https://www.worldathletics.org/athletes/athlete=115602, year 13, perf 13, place 13, date 13
athlete https://www.worldathletics.org/athletes/athlete=115424, year 18, perf 18, place 18, date 18
athlete https://www.worldathletics.org/athletes/athlete=221143, year 24, perf 24, place 24, date 24
athlete https://www.worldathletics.org/athletes/athlete=80551, year 34, perf 34, place 34, date 34
athlete https://www.worldathletics.org/athletes/athlete=200601, year 40, perf 40, place 40, date 40
athlete https://www.worldathletics.org/athletes/athlete=258886, year 47, perf 47, place 47, date 47
athlete https://www.worldathletics.org/athletes/athlete=194565, year 58, perf 58, place 58, date 58
athlete https://www.worldathletics.org/athletes/athlete=60762, year 63, perf 63, place 63, date 63
athlete https://www.worldathletics.org/athletes/athlete=195490, year 73, perf 73, place 73, date 73
athlet

athlete https://www.worldathletics.org/athletes/athlete=179866, year 533, perf 533, place 533, date 533
athlete https://www.worldathletics.org/athletes/athlete=115764, year 539, perf 539, place 539, date 539
athlete https://www.worldathletics.org/athletes/athlete=61790, year 559, perf 559, place 559, date 559
athlete https://www.worldathletics.org/athletes/athlete=223876, year 560, perf 560, place 560, date 560
athlete https://www.worldathletics.org/athletes/athlete=272230, year 571, perf 571, place 571, date 571
athlete https://www.worldathletics.org/athletes/athlete=188325, year 573, perf 573, place 573, date 573
athlete https://www.worldathletics.org/athletes/athlete=250664, year 579, perf 579, place 579, date 579
athlete https://www.worldathletics.org/athletes/ireland/marie-murphy-rollins-014429581, year 581, perf 581, place 581, date 581
athlete https://www.worldathletics.org/athletes/athlete=181744, year 583, perf 583, place 583, date 583
athlete https://www.worldathletics.org/at

athlete https://www.worldathletics.org/athletes/athlete=203411, year 1079, perf 1079, place 1079, date 1079
athlete https://www.worldathletics.org/athletes/athlete=131219, year 1083, perf 1083, place 1083, date 1083
athlete https://www.worldathletics.org/athletes/athlete=246176, year 1086, perf 1086, place 1086, date 1086
athlete https://www.worldathletics.org/athletes/athlete=259297, year 1095, perf 1095, place 1095, date 1095
athlete https://www.worldathletics.org/athletes/athlete=259298, year 1099, perf 1099, place 1099, date 1099
athlete https://www.worldathletics.org/athletes/athlete=174994, year 1107, perf 1107, place 1107, date 1107
athlete https://www.worldathletics.org/athletes/athlete=311814, year 1116, perf 1116, place 1116, date 1116
athlete https://www.worldathletics.org/athletes/athlete=201703, year 1123, perf 1123, place 1123, date 1123
athlete https://www.worldathletics.org/athletes/athlete=136302, year 1134, perf 1134, place 1134, date 1134
athlete https://www.worldath

In [146]:
df = pd.DataFrame({"athlete": athlete, "year": year, "performance": performance, "place":place, "date":date})
df.to_csv(r"C:\Users\BrechtDewilde\Documents\Github\Predicting-the-olympic-games\marathon\Data c&c\R\Data\record history\female_records.csv", index = False)

In [ ]:
df = pd.DataFrame({"Name": col0 , "Performance": col1, "Place": col2, "Date": col3})
df.to_csv(r"C:\Users\BrechtDewilde\Documents\Github\Predicting-the-olympic-games\IDA code\Marathon\Data\scraped\world_athletics\world_athletics.csv", index = False)